# Epoching 
    1. Rejecting all the epochs wiht peak-peak value > 200uV;
    2. Preserve (-0.5, 1) in epochs for latter analysis;
    3. Using (-0.5, 0) as baseline for correction;
    4. Trigger values have been updated to the true trigger values;
    5. Save report for each subject including: a) Basic Info, 2) Events; 3) Epochs. 
## Outputs:
    1. Epoch data: in subject folder (derivative/epochs/sub-xxx/sub-xxx_task-xxxx_desc-epoched_epo.fif)  
    2. Reports: in both subject folder (derivative/epochs/sub-xxx/sub-xxx_task-xxxx_desc-epoched_epo.fif) and derivative/epochs/sub-xxx_report.html
## Observations:
    1. Running with docker on windows (WLS2 backend) several times but fail, running ```epoching_win.py``` (in the same folder) instead to do the epoching.

In [1]:
import os
from pathlib import Path
import mne
from mne_bids import BIDSPath, read_raw_bids, write_raw_bids, print_dir_tree, make_report, print_dir_tree, inspect_dataset
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs, corrmap
import numpy as np
import pandas as pd
#%matplotlib inline

# project folders
bids_dir = Path("/data/raw")
codes_dir = Path("/codes")
preproc_dir = Path("/output/preproc")
epoch_dir = Path("/output/epochs")

# subject list
sub_list = [f"{a:03}" for a in range(1,34,1)]

# subject raw eeg file dict
sub_eeg_list = [(bids_dir / ("sub-"+x) / "eeg" / ("sub-"+x+"_task-xxxx_eeg.vhdr")) for x in sub_list]
sub_raw_eeg_dict = dict(zip(sub_list, sub_eeg_list))
# subject preproc eeg file dict
sub_eeg_file_list = [ (preproc_dir / ("sub-"+x) / ("sub-"+x+"_task-xxxx_desc-preproc_eeg.fif")) for x in sub_list]
sub_eeg_dict = dict(zip(sub_list , sub_eeg_file_list))
# subject epoch file dict
sub_epoch_file_list = [ (epoch_dir / ("sub-"+x) / ("sub-"+x+"_task-xxxx_desc-epoched_epo.fif")) for x in sub_list]
sub_epoch_dict = dict(zip(sub_list , sub_epoch_file_list))
# subject report file dict
sub_report_file_list = [ (epoch_dir / ("sub-"+x+"_report.html")) for x in sub_list]
sub_report_dict = dict(zip(sub_list , sub_report_file_list))
# subject report file in subject folder dict
sub_folder_report_file_list = [ (epoch_dir / ("sub-"+x) / ("sub-"+x+"_report.html")) for x in sub_list]
sub_folder_report_dict = dict(zip(sub_list , sub_folder_report_file_list))

# overall triger data
trigger_data_dir = codes_dir / "tab_data" / "TriggerTable.csv"
trigger_df = pd.read_csv(trigger_data_dir, sep=',');

In [ ]:
# Epoching block
# Epoch rejection threshold: 200 µV, comparatively conservative
reject_criteria = dict(eeg=200e-6) 
# Using -0.5~0 as baseline for activation correction.
baseline_th = (-0.5, 0)
# preserving larger time span (-0.5, 1) to include baselien, N1 (150~200ms) and 300~500ms
T_MIN = -0.5
T_MAX = 1

# full trigger data
trigger_codes=trigger_df.trigger.unique();
full_trigger_dict=dict(zip(["Stimulus/"+str(x) for x in trigger_codes], trigger_codes))

# loop over for epoching
for sub_id in sub_list:
    ## read eeg data sub_list
    print("sub-"+sub_id+": reading preproc eeg...")
    eeg = mne.io.read_raw_fif(sub_eeg_dict[sub_id], preload=True)
    
    # updating events coding
    events_from_annot, event_dict = mne.events_from_annotations(eeg, event_id=full_trigger_dict)
    events_keys=list(set(event_dict.keys()) & set(full_trigger_dict.keys()))
    updated_events_dict={}
    for x in events_keys:
        updated_events_dict[x]=full_trigger_dict[x]
    # epochs rejection
    print("sub-"+sub_id+": epoching...")
    epochs = mne.Epochs(eeg, events_from_annot, event_id=updated_events_dict, baseline = baseline_th,
                        tmin=T_MIN, tmax=T_MAX, reject=reject_criteria,
                        preload=True, on_missing="warn");
    epochs.add_annotations_to_metadata(overwrite=True)
    epoch_sub_dir = epoch_dir/ ("sub-"+sub_id)
    if (not epoch_sub_dir.is_dir()):
        os.mkdir(epoch_sub_dir) 
    epochs.save(sub_epoch_dict[sub_id], overwrite=True)
    # Generate report
    report = mne.Report(title=sub_id+' Report')
    report.add_raw(raw=eeg, title='Basic Info', psd=True)   # omit PSD plot
    report.add_events(events=events_from_annot, title='Events from annotations', sfreq=eeg.info["sfreq"])
    report.add_epochs(epochs=epochs, title="Epochs")
    report.save(sub_report_dict[sub_id], overwrite=True)
    report.save(sub_folder_report_dict[sub_id], overwrite=True)

sub-001: reading preproc eeg...
Opening raw data file /output/preproc/sub-001/sub-001_task-xxxx_desc-preproc_eeg.fif...
    Range : 0 ... 2541055 =      0.000 ...  4962.998 secs
Ready.
Reading 0 ... 2541055  =      0.000 ...  4962.998 secs...
Used Annotations descriptions: ['Stimulus/1030', 'Stimulus/1031', 'Stimulus/1039', 'Stimulus/1040', 'Stimulus/1041', 'Stimulus/1049', 'Stimulus/1110', 'Stimulus/1111', 'Stimulus/1119', 'Stimulus/1120', 'Stimulus/1121', 'Stimulus/1129', 'Stimulus/2030', 'Stimulus/2031', 'Stimulus/2039', 'Stimulus/2040', 'Stimulus/2041', 'Stimulus/2049', 'Stimulus/2091', 'Stimulus/2110', 'Stimulus/2111', 'Stimulus/2119', 'Stimulus/2120', 'Stimulus/2121', 'Stimulus/2129']
sub-001: epoching...
Not setting metadata
1200 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1200 events and 769 original time points ...
    Rejecting  epoch based on EEG : ['Fp1', 'AF7', 'F7']
    Rejecting  epoch bas

/tmp/ipykernel_70/3585558340.py:39: RuntimeWarning: More events than default colors available. You should pass a list of unique colors.
  report.add_events(events=events_from_annot, title='Events from annotations', sfreq=eeg.info["sfreq"])


    Using multitaper spectrum estimation with 7 DPSS windows
Saving report to : /output/epochs/sub-001_report.html
Saving report to : /output/epochs/sub-001/sub-001_report.html
sub-002: reading preproc eeg...
Opening raw data file /output/preproc/sub-002/sub-002_task-xxxx_desc-preproc_eeg.fif...
    Range : 0 ... 1840127 =      0.000 ...  3593.998 secs
Ready.
Reading 0 ... 1840127  =      0.000 ...  3593.998 secs...
Used Annotations descriptions: ['Stimulus/1030', 'Stimulus/1031', 'Stimulus/1039', 'Stimulus/1040', 'Stimulus/1041', 'Stimulus/1049', 'Stimulus/1110', 'Stimulus/1111', 'Stimulus/1119', 'Stimulus/1120', 'Stimulus/1121', 'Stimulus/1129', 'Stimulus/2030', 'Stimulus/2031', 'Stimulus/2039', 'Stimulus/2040', 'Stimulus/2041', 'Stimulus/2049', 'Stimulus/2110', 'Stimulus/2111', 'Stimulus/2119', 'Stimulus/2120', 'Stimulus/2121', 'Stimulus/2129']
sub-002: epoching...
Not setting metadata
1200 matching events found
Applying baseline correction (mode: mean)
0 projection items activated


/tmp/ipykernel_70/3585558340.py:39: RuntimeWarning: More events than default colors available. You should pass a list of unique colors.
  report.add_events(events=events_from_annot, title='Events from annotations', sfreq=eeg.info["sfreq"])


    Using multitaper spectrum estimation with 7 DPSS windows
Saving report to : /output/epochs/sub-002_report.html
Saving report to : /output/epochs/sub-002/sub-002_report.html
sub-003: reading preproc eeg...
Opening raw data file /output/preproc/sub-003/sub-003_task-xxxx_desc-preproc_eeg.fif...
    Range : 0 ... 1991679 =      0.000 ...  3889.998 secs
Ready.
Reading 0 ... 1991679  =      0.000 ...  3889.998 secs...
Used Annotations descriptions: ['Stimulus/1030', 'Stimulus/1031', 'Stimulus/1039', 'Stimulus/1040', 'Stimulus/1041', 'Stimulus/1049', 'Stimulus/1110', 'Stimulus/1111', 'Stimulus/1119', 'Stimulus/1120', 'Stimulus/1121', 'Stimulus/1129', 'Stimulus/2030', 'Stimulus/2031', 'Stimulus/2039', 'Stimulus/2040', 'Stimulus/2041', 'Stimulus/2049', 'Stimulus/2110', 'Stimulus/2111', 'Stimulus/2119', 'Stimulus/2120', 'Stimulus/2121', 'Stimulus/2129']
sub-003: epoching...
Not setting metadata
1200 matching events found
Applying baseline correction (mode: mean)
0 projection items activated


...

/tmp/ipykernel_70/3585558340.py:39: RuntimeWarning: More events than default colors available. You should pass a list of unique colors.
  report.add_events(events=events_from_annot, title='Events from annotations', sfreq=eeg.info["sfreq"])


    Using multitaper spectrum estimation with 7 DPSS windows
Saving report to : /output/epochs/sub-017_report.html
Saving report to : /output/epochs/sub-017/sub-017_report.html
sub-018: reading preproc eeg...
Opening raw data file /output/preproc/sub-018/sub-018_task-xxxx_desc-preproc_eeg.fif...
    Range : 0 ... 2240511 =      0.000 ...  4375.998 secs
Ready.
Reading 0 ... 2240511  =      0.000 ...  4375.998 secs...
Used Annotations descriptions: ['Stimulus/1030', 'Stimulus/1031', 'Stimulus/1039', 'Stimulus/1040', 'Stimulus/1041', 'Stimulus/1049', 'Stimulus/1091', 'Stimulus/1099', 'Stimulus/1110', 'Stimulus/1111', 'Stimulus/1119', 'Stimulus/1120', 'Stimulus/1121', 'Stimulus/1129', 'Stimulus/1199', 'Stimulus/2030', 'Stimulus/2031', 'Stimulus/2039', 'Stimulus/2040', 'Stimulus/2041', 'Stimulus/2049', 'Stimulus/2090', 'Stimulus/2099', 'Stimulus/2110', 'Stimulus/2111', 'Stimulus/2119', 'Stimulus/2120', 'Stimulus/2121', 'Stimulus/2129', 'Stimulus/2190', 'Stimulus/2199']
sub-018: epoching...


/tmp/ipykernel_70/3585558340.py:39: RuntimeWarning: More events than default colors available. You should pass a list of unique colors.
  report.add_events(events=events_from_annot, title='Events from annotations', sfreq=eeg.info["sfreq"])


    Using multitaper spectrum estimation with 7 DPSS windows
Saving report to : /output/epochs/sub-018_report.html
Saving report to : /output/epochs/sub-018/sub-018_report.html
sub-019: reading preproc eeg...
Opening raw data file /output/preproc/sub-019/sub-019_task-xxxx_desc-preproc_eeg.fif...
    Range : 0 ... 2308095 =      0.000 ...  4507.998 secs
Ready.
Reading 0 ... 2308095  =      0.000 ...  4507.998 secs...
Used Annotations descriptions: ['Stimulus/1030', 'Stimulus/1031', 'Stimulus/1039', 'Stimulus/1040', 'Stimulus/1041', 'Stimulus/1049', 'Stimulus/1110', 'Stimulus/1111', 'Stimulus/1119', 'Stimulus/1120', 'Stimulus/1121', 'Stimulus/1129', 'Stimulus/1191', 'Stimulus/2030', 'Stimulus/2031', 'Stimulus/2039', 'Stimulus/2040', 'Stimulus/2041', 'Stimulus/2049', 'Stimulus/2110', 'Stimulus/2111', 'Stimulus/2119', 'Stimulus/2120', 'Stimulus/2121', 'Stimulus/2129', 'Stimulus/2199']
sub-019: epoching...
Not setting metadata
1200 matching events found
Applying baseline correction (mode: m

/tmp/ipykernel_70/3585558340.py:39: RuntimeWarning: More events than default colors available. You should pass a list of unique colors.
  report.add_events(events=events_from_annot, title='Events from annotations', sfreq=eeg.info["sfreq"])
